In [30]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [9]:
nppes = pd.read_csv("nppes_with_taxonomy.csv")

In [10]:
nppes.head(10)

,npi,entity_code,org_name,last_name,first_name,middle,prefix,suffix,credential,address1,address2,city,state,postal_code,primary_taxonomy
0,1679576722,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,3500 CENTRAL AVE,NaN,KEARNEY,NE,688472944.0,207X00000X
1,1588667638,1.0,NaN,PILCHER,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,322044736.0,207RC0000X
2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,NaN,3418 VILLAGE DR,NaN,FAYETTEVILLE,NC,283044552.0,251G00000X
3,1215930367,1.0,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,M.D.,17323 RED OAK DR,NaN,HOUSTON,TX,770901243.0,207RH0003X
4,1023011178,2.0,COLLABRIA CARE,NaN,NaN,NaN,NaN,NaN,NaN,414 S JEFFERSON ST,NaN,NAPA,CA,945594515.0,251G00000X
5,1932102084,1.0,NaN,ADUSUMILLI,RAVI,K,NaN,NaN,MD,2940 N MCCORD RD,NaN,TOLEDO,OH,436151753.0,207RC0000X
6,1841293990,1.0,NaN,WORTSMAN,SUSAN,NaN,NaN,NaN,MA-CCC,425 E 25TH ST,NaN,NEW YORK,NY,100102547.0,231H00000X
7,1750384806,1.0,NaN,BISBEE,ROBERT,NaN,DR.,NaN,MD,808 JOLIET AVE UNIT 120,NaN,LUBBOCK,TX,794151148.0,207R00000X
8,1669475711,1.0,NaN,SUNG,BIN,SHENG,NaN,NaN,M. D.,7629 TIKI DR,NaN,FULSHEAR,TX,774411548.0,208000000X
9,1578566626,1.0,NaN,KUIPERS,WARREN,D.,NaN,NaN,MD,1205 S 7TH AVE,NaN,PHOENIX,AZ,850073913.0,207Q00000X


In [32]:
hop_proj=sqlite3.connect('hop_proj.sqlite')
for chunk in tqdm(pd.read_csv('nppes_with_taxonomy.csv', dtype = {'postal_code':str}, chunksize = 1000)):
    chunk['postal_code'] = chunk['postal_code'].str.slice(stop=5)
    chunk.to_sql('nppes', hop_proj, if_exists = 'append', index = False)
for chunk in tqdm(pd.read_csv('nucc_taxonomy_220.csv', chunksize = 1000)):
    chunk.to_sql('taxonomy', hop_proj, if_exists = 'append', index = False)
for chunk in tqdm(pd.read_csv('ZIP_CBSA_122021.csv', chunksize = 1000, )):
    chunk['zip'] = chunk['zip'].str.slice(start = 1, stop = 6)
    chunk.to_sql('cbsa', hop_proj, if_exists = 'append', index = False)
for chunk in tqdm(pd.read_csv('DocGraph_Hop_Teaming_2018.csv',
    dtype = {'zip':str},
    chunksize = 1000)):
    chunk = chunk[(chunk['transaction_count'] >= 50) & chunk['average_day_wait'] < 50]
    chunk.to_sql('hop', hop_proj, if_exists = 'append', index = False)
                                  

0it [00:00, ?it/s]

KeyboardInterrupt: 

In [34]:
for chunk in tqdm(pd.read_csv('DocGraph_Hop_Teaming_2018.csv',
    dtype = {'zip':str},
    chunksize = 1000)):
    chunk = chunk[(chunk['transaction_count'] >= 50) & chunk['average_day_wait'] < 50]

0it [00:00, ?it/s]

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
217911000,1417901612,1497933162,24,25,94.400,82.175
217911001,1417905522,1497933162,39,40,98.850,88.031
217911002,1417901091,1497933162,544,659,56.725,58.623
217911003,1417905969,1497933162,12,19,50.000,66.775
217911004,1417905092,1497933162,16,17,115.765,106.966
...,...,...,...,...,...,...
217911303,1417903683,1497940662,51,51,104.706,83.400
217911304,1417909367,1497940662,131,136,21.051,30.066
217911305,1417909029,1497940662,25,26,41.077,58.280
217911306,1417916610,1497940662,11,11,64.545,87.041
